In [1]:
"""
Created on Wed Aug  4 17:50:06 2021

@author: amol
"""

# set TF GPU memory growth so that it doesn't hog everything at once
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

import os
import numpy as np
from main import *
debug_mode = True  # faster experiments for debugging

# util for getting objects' fields' names
field_names = lambda x: list(vars(x).keys())

In [2]:
#Prepare the DataFrame that will be used downstream
dp = DataPreparer()
dp.createPCAs()
dp.sparsePCAs()
dp.zmixOrthogonalPCAs()
df = dp.getDataframe()

# currently passing dp eventually we want to abstract all the constants into 1 class
dm = DataManager(df, dp)

In [3]:
"""
bestModel, experimentSettings = exprExec.modelFactory.openBestModel()
dm.createTrainTestData(experimentSettings['dataSetMethod'], experimentSettings['noOfCpv'],
                       experimentSettings['ipscaler'], experimentSettings['opscaler'])
dm.input_data_cols
"""

"\nbestModel, experimentSettings = exprExec.modelFactory.openBestModel()\ndm.createTrainTestData(experimentSettings['dataSetMethod'], experimentSettings['noOfCpv'],\n                       experimentSettings['ipscaler'], experimentSettings['opscaler'])\ndm.input_data_cols\n"

In [4]:
'''
Run the Model Experiments
'''
model = 'PCDNN_V2'
assert model in ['PCDNN_V2', 'PCDNN_V1', 'SIMPLE_DNN', 'GP'] # valid possible models
exprExec=run_model_experiments(dm, models=model, debug_mode=False)

/home/dwyerdei/rom_project2/src/experiment_executor/pcdnn_v2_experiment_executor.py:33: UserWarning: manually overriding n models to 1! change this!
  warnings.warn('manually overriding n models to 1! change this!')


Parent DNNModelFactory Instantiated
=================== randomequaltraintestsplit ===================
------------------ AllSpeciesAndZmix ------------------
--------------------self.build_and_compile_pcdnn_v2_model----------------------
53 4 Y Y Y


KeyboardInterrupt: 

In [3]:
""" prepare SimpleDNN for loading (from prior experiments) """

exprExec = DNNExperimentExecutor()
exprExec.debug_mode = debug_mode

exprExec.setModelFactory(SimpleDNNModelFactory())

Parent DNNModelFactory Instantiated


In [4]:
""" prepare PCDNNV1 for loading (from prior experiments) """

exprExec = PCDNNV1ExperimentExecutor()
exprExec.debug_mode = debug_mode

exprExec.setModelFactory(PCDNNV1ModelFactory())

Parent DNNModelFactory Instantiated


In [5]:
""" prepare PCDNNV2 for loading (from prior experiments) """

exprExec = PCDNNV2ExperimentExecutor()
exprExec.debug_mode = debug_mode

exprExec.setModelFactory(PCDNNV2ModelFactory())

Parent DNNModelFactory Instantiated


/home/dwyerdei/rom_project2/src/experiment_executor/pcdnn_v2_experiment_executor.py:33: UserWarning: manually overriding n models to 1! change this!
  warnings.warn('manually overriding n models to 1! change this!')


## Rapid Model Testing: 
### (requires setting up PCDNNV2 for loading)

In [ ]:
# fix seeds
import random
import numpy as np
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

dataType = 'randomequaltraintestsplit' #'frameworkincludedtrainexcludedtest'
inputType = 'AllSpeciesAndZmix'
dataSetMethod = f'{inputType}_{dataType}'
opscaler = "MinMaxScaler"# 'PositiveLogNormal'
ZmixPresent = 'Y'
concatenateZmix = 'Y'
kernel_constraint = 'Y'
kernel_regularizer = 'Y'
activity_regularizer = 'Y'
noOfCpv = 4
noOfNeurons = 53

exprExec.modelFactory.loss='mae'
exprExec.modelFactory.activation_func='relu'
exprExec.modelFactory.dropout_rate=0.5
exprExec.debug_mode = False
exprExec.epochs_override = 100
exprExec.batch_size = 32
exprExec.n_models_override = 1

# initialize experiment executor...
exprExec.dm = dm
exprExec.df_experimentTracker = pd.DataFrame()
exprExec.modelType = 'PCDNNV2'

history = exprExec.executeSingleExperiment(noOfNeurons,dataSetMethod,dataType,inputType,ZmixPresent=ZmixPresent,
                                           noOfCpv=noOfCpv,concatenateZmix=concatenateZmix,kernel_constraint=kernel_constraint,
                                           kernel_regularizer=kernel_regularizer,activity_regularizer=activity_regularizer,
                                           opscaler=opscaler)

--------------------self.build_and_compile_pcdnn_v2_model----------------------
53 4 Y Y Y
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 species_input (InputLayer)     [(None, 53)]         0           []                               
                                                                                                  
 zmix (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 linear_embedding (Functional)  (None, 4)            216         ['species_input[0][0]']          
                                                                                                  
 concatenated_zmix_linear_embed  (None, 5)           0           ['zmix[0][0]',                   
 

Epoch 7/100
257/257 [==============================] - 3s 13ms/step - loss: 0.0103 - mae: 0.0082 - mse: 3.7735e-04 - exp_mse_mag: -3.3533 - exp_mae_mag: -2.0177 - exp_R2: 0.9627 - R2: 0.9693 - val_loss: 0.0085 - val_mae: 0.0067 - val_mse: 3.1996e-04 - val_exp_mse_mag: -3.5494 - val_exp_mae_mag: -2.1045 - val_exp_R2: 0.9751 - val_R2: 0.9772
Epoch 8/100
257/257 [==============================] - 3s 14ms/step - loss: 0.0091 - mae: 0.0065 - mse: 2.5519e-04 - exp_mse_mag: -3.5384 - exp_mae_mag: -2.1113 - exp_R2: 0.9781 - R2: 0.9816 - val_loss: 0.0088 - val_mae: 0.0052 - val_mse: 2.6649e-04 - val_exp_mse_mag: -3.6995 - val_exp_mae_mag: -2.2174 - val_exp_R2: 0.9811 - val_R2: 0.9834
Epoch 9/100
257/257 [==============================] - 3s 14ms/step - loss: 0.0091 - mae: 0.0067 - mse: 2.8033e-04 - exp_mse_mag: -3.5304 - exp_mae_mag: -2.1079 - exp_R2: 0.9744 - R2: 0.9785 - val_loss: 0.0110 - val_mae: 0.0086 - val_mse: 4.3644e-04 - val_exp_mse_mag: -3.2947 - val_exp_mae_mag: -1.9870 - val_exp_R2

Epoch 31/100
257/257 [==============================] - 3s 14ms/step - loss: 0.0078 - mae: 0.0042 - mse: 1.3356e-04 - exp_mse_mag: -3.9374 - exp_mae_mag: -2.3131 - exp_R2: 0.9886 - R2: 0.9904 - val_loss: 0.0136 - val_mae: 0.0087 - val_mse: 3.8007e-04 - val_exp_mse_mag: -3.3686 - val_exp_mae_mag: -1.9955 - val_exp_R2: 0.9592 - val_R2: 0.9609
Epoch 32/100
257/257 [==============================] - 3s 14ms/step - loss: 0.0079 - mae: 0.0046 - mse: 1.6122e-04 - exp_mse_mag: -3.8399 - exp_mae_mag: -2.2668 - exp_R2: 0.9865 - R2: 0.9886 - val_loss: 0.0072 - val_mae: 0.0051 - val_mse: 2.1979e-04 - val_exp_mse_mag: -3.7303 - val_exp_mae_mag: -2.2172 - val_exp_R2: 0.9836 - val_R2: 0.9853
Epoch 33/100
257/257 [==============================] - 3s 14ms/step - loss: 0.0064 - mae: 0.0042 - mse: 1.3560e-04 - exp_mse_mag: -3.9453 - exp_mae_mag: -2.3195 - exp_R2: 0.9875 - R2: 0.9892 - val_loss: 0.0053 - val_mae: 0.0033 - val_mse: 1.5904e-04 - val_exp_mse_mag: -4.1999 - val_exp_mae_mag: -2.4303 - val_exp

Epoch 55/100
257/257 [==============================] - 3s 14ms/step - loss: 0.0057 - mae: 0.0034 - mse: 1.0340e-04 - exp_mse_mag: -4.1439 - exp_mae_mag: -2.4135 - exp_R2: 0.9889 - R2: 0.9904 - val_loss: 0.0054 - val_mae: 0.0038 - val_mse: 1.4574e-04 - val_exp_mse_mag: -4.0914 - val_exp_mae_mag: -2.3711 - val_exp_R2: 0.9895 - val_R2: 0.9902
Epoch 56/100
257/257 [==============================] - 3s 14ms/step - loss: 0.0056 - mae: 0.0036 - mse: 1.1160e-04 - exp_mse_mag: -4.1162 - exp_mae_mag: -2.4016 - exp_R2: 0.9915 - R2: 0.9930 - val_loss: 0.0094 - val_mae: 0.0067 - val_mse: 3.1626e-04 - val_exp_mse_mag: -3.3696 - val_exp_mae_mag: -2.0748 - val_exp_R2: 0.9767 - val_R2: 0.9815
Epoch 57/100
257/257 [==============================] - 3s 14ms/step - loss: 0.0051 - mae: 0.0031 - mse: 9.3527e-05 - exp_mse_mag: -4.1601 - exp_mae_mag: -2.4363 - exp_R2: 0.9916 - R2: 0.9930 - val_loss: 0.0053 - val_mae: 0.0029 - val_mse: 1.3460e-04 - val_exp_mse_mag: -4.4251 - val_exp_mae_mag: -2.4997 - val_exp

Epoch 79/100
257/257 [==============================] - 3s 14ms/step - loss: 0.0044 - mae: 0.0029 - mse: 7.3783e-05 - exp_mse_mag: -4.2591 - exp_mae_mag: -2.4736 - exp_R2: 0.9922 - R2: 0.9935 - val_loss: 0.0037 - val_mae: 0.0025 - val_mse: 1.0752e-04 - val_exp_mse_mag: -4.4282 - val_exp_mae_mag: -2.5588 - val_exp_R2: 0.9935 - val_R2: 0.9942
Epoch 80/100
257/257 [==============================] - 3s 14ms/step - loss: 0.0049 - mae: 0.0030 - mse: 8.2739e-05 - exp_mse_mag: -4.2215 - exp_mae_mag: -2.4571 - exp_R2: 0.9937 - R2: 0.9947 - val_loss: 0.0055 - val_mae: 0.0039 - val_mse: 1.8674e-04 - val_exp_mse_mag: -3.7936 - val_exp_mae_mag: -2.3220 - val_exp_R2: 0.9879 - val_R2: 0.9902
Epoch 81/100
257/257 [==============================] - 3s 14ms/step - loss: 0.0050 - mae: 0.0029 - mse: 8.0106e-05 - exp_mse_mag: -4.2543 - exp_mae_mag: -2.4798 - exp_R2: 0.9945 - R2: 0.9953 - val_loss: 0.0075 - val_mae: 0.0032 - val_mse: 1.3039e-04 - val_exp_mse_mag: -3.9742 - val_exp_mae_mag: -2.4087 - val_exp

## Results Plotting & Analysis

In [ ]:
loss = 727684307.3417714 # Update Me!

def print_scientific_notation(number):
    power = int(np.log(number)/np.log(10))
    print(f"Scientific Notation: {(loss/10**power)}*10^{power}")
print_scientific_notation(loss)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
plt.plot(np.maximum(history.history['val_R2'][30:], 0))
plt.title('val_R^2 vs epochs')
plt.ylabel('percent')
plt.xlabel('epochs')
plt.show()

In [ ]:
import model_analyzer.model_analysis as model_analysis
import importlib; importlib.reload(model_analysis)

bestModel, experimentSettings = exprExec.modelFactory.openBestModel()
dm.createTrainTestData(experimentSettings['dataSetMethod'],experimentSettings['noOfCpv'], experimentSettings['ipscaler'], experimentSettings['opscaler'])

print(f'\nexperimentSettings: {experimentSettings}')
print(f'\nbestModel.input_shape: {bestModel.input_shape}')
inspector = model_analysis.ModelInspector(exprExec.modelFactory, dm)

In [ ]:
n_repeats = 5 if debug_mode else 20
inspector.plot_permutation_feature_importance(n_repeats=n_repeats)

In [ ]:
inspector.plot_partial_dependence()

### Print & Record Linear Embeddings Output (NOTE: Deprecated)

In [ ]:
import numpy as np

def inspect_PCA(X):
    PCA_dict = {}
    PCA_dict['X'] = X
    PCA_dict['Var'] = PCA_dict['X'].var(axis=0)
    
    # sort by explained variance
    sort_idx = np.argsort(PCA_dict['Var'])[::-1]
    PCA_dict['X'] = PCA_dict['X'][:, sort_idx]
    PCA_dict['Var'] = PCA_dict['Var'][sort_idx]
    PCA_dict['CumVar'] = np.add.accumulate(PCA_dict['Var'])
    return PCA_dict


X, Y, rom, zmix = dm.getAllData()
FullDataset = inspect_PCA(X)
try:    
    linearAutoEncoder = exprExec.modelFactory.getLinearEncoder()
    X = linearAutoEncoder.predict(FullDataset['X'])
    DNN_PCA = inspect_PCA(X)
except KeyError:
    print('No Linear Auto Encoder!')

In [ ]:
#dm.createDataset
import matplotlib.pyplot as plt
import copy
dm_PurePCA = copy.deepcopy(dm)
dm_PurePCA.createTrainTestData(dataSetMethod='PurePCA_randomequalflamesplit',
                               numCpvComponents=2, ipscaler=None, opscaler=None)

X,Y,rom,zmix = dm_PurePCA.getAllData()
PurePCA = inspect_PCA(X)
print(PurePCA['Var'])
print(DNN_PCA['Var'])

plt.plot(PurePCA['CumVar']/FullDataset['Var'].sum(), color='r')
plt.plot(DNN_PCA['CumVar']/FullDataset['Var'].sum(), color='b')
plt.title('PCA Total Variance Comparison')